In [145]:
import torch 
from torch import nn
from torchinfo import summary

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

import onnx
import onnxruntime

%matplotlib inline

In [82]:
print(' Cuda:', torch.cuda.is_available(), "\n" ,'Apple MPS:', torch.backends.mps.is_available())

 Cuda: False 
 Apple MPS: True


#### Full pytorch workflow with MNIST fashion dataset used in the Pytorch learning path at Microsoft

In [123]:
# GPU use
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # prefers GPU over CPU

# Increases accuracy to 71 % compared to transform = ToTensor(), which delivers 41 %.
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform)

In [124]:
batch_size = 4

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [125]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device) # to device does not work here. see next cell
summary(model,input_size=(1,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Linear: 2-1                       [1, 512]                  401,920
│    └─ReLU: 2-2                         [1, 512]                  --
│    └─Linear: 2-3                       [1, 512]                  262,656
│    └─ReLU: 2-4                         [1, 512]                  --
│    └─Linear: 2-5                       [1, 10]                   5,130
│    └─ReLU: 2-6                         [1, 10]                   --
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
Total mult-adds (M): 0.67
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.68
Estimated Total Size (MB): 2.69

In [135]:
model = NeuralNetwork().to(device)

In [136]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3

# the addition of momentum = .9 improves the accuracy from 80 % to 84 % in 1 epoch
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=.9) 

In [137]:
def train_loop(device, dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):  
        # possible representation of (X, y) with more lines of code.      
        # is just (X, y) = data an in additional line
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad() # zero your gradients for every batch!
        loss.backward()
        optimizer.step() # gradient descent

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(device, dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad(): # disables gradient calculation
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### GPU runtime

In [138]:
epochs = 1
def main():
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(device, train_dataloader, model, loss_fn, optimizer)
        test_loop(device, test_dataloader, model, loss_fn)
    print("Done")
main()

Epoch 1
-------------------------------
loss: 2.296759  [    0/60000]
loss: 0.561553  [ 4000/60000]
loss: 0.322416  [ 8000/60000]
loss: 1.399738  [12000/60000]
loss: 2.023489  [16000/60000]
loss: 1.163917  [20000/60000]
loss: 0.661505  [24000/60000]
loss: 2.435496  [28000/60000]
loss: 1.831609  [32000/60000]
loss: 0.149050  [36000/60000]
loss: 1.230498  [40000/60000]
loss: 0.364594  [44000/60000]
loss: 0.635265  [48000/60000]
loss: 0.582289  [52000/60000]
loss: 0.616623  [56000/60000]
Test Error: 
 Accuracy: 71.9%, Avg loss: 0.223090 

Done


### CPU runtime
https://discuss.pytorch.org/t/sequential-throughput-of-gpu-execution/156303

In [139]:
device = torch.device("mps" if not torch.backends.mps.is_available() else "cpu") # prefers CPU over GPU
epochs = 1
def main():
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(device, train_dataloader, model, loss_fn, optimizer)
        test_loop(device, test_dataloader, model, loss_fn)
    print("Done")
main()

Epoch 1
-------------------------------
loss: 1.899077  [    0/60000]
loss: 0.108631  [ 4000/60000]
loss: 0.132975  [ 8000/60000]
loss: 1.276919  [12000/60000]
loss: 2.291896  [16000/60000]
loss: 1.218495  [20000/60000]
loss: 0.570299  [24000/60000]
loss: 1.888932  [28000/60000]
loss: 1.804555  [32000/60000]
loss: 0.087260  [36000/60000]
loss: 1.171256  [40000/60000]
loss: 0.306714  [44000/60000]
loss: 0.595607  [48000/60000]
loss: 0.579024  [52000/60000]
loss: 0.029249  [56000/60000]
Test Error: 
 Accuracy: 75.7%, Avg loss: 0.164483 

Done


### Saving the model

In [130]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [131]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('data/model.pth'))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)

In [140]:
input_image = torch.zeros((1,28,28))
onnx_model = 'data/model.onnx'
torch.onnx.export(model, input_image, onnx_model)

In [141]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
x, y = test_data[0][0], test_data[0][1]

In [ ]:
onnx_model = onnx.load("fashion_mnist_model.onnx")
onnx.checker.check_model(onnx_model)

In [146]:
session = onnxruntime.InferenceSession(onnx_model, None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

result = session.run([output_name], {input_name: x.numpy()})
predicted, actual = classes[result[0][0].argmax(0)], classes[y]
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


## Loading your own images

In most of the practical applications, you would have your own images located on disk that you want to use to train your neural network. In this case, you need to load them into PyTorch tensors. 

One of the ways to do that is to use one of the Python libraries for image manipulation, such as *Open CV*, or *PIL/Pillow*, or *imageio*. Once you load your image into numpy array, you can easily convert it to tensors. 

> It is important to make sure that all values are scaled to the range [0..1] before you pass them to a neural network - it is the usual convention for data preparation, and all default weight initializations in neural networks are designed to work with this range. `ToTensor` transform that we have seen above automatically scales PIL/numpy images with integer pixel values into [0..1] range.

Even better approach is to use functionality in **Torchvision** library, namely `ImageFolder`. It does all the preprocessing steps automatically, and also assigns labels to images according to the directory structure. We will see the example of using `ImageFolder` later in this course, once we start classifying our own cats and dogs images.

> It is important to note that all images should be scaled to the same size. If your original images have different aspect ratios, you need to decide how to handle this scaling - either by cropping images, or by padding extra space.

## Takeaway

Neural networks work with tensors, and before training any models we need to convert our dataset into a set of tensors. This will often require reshaping. We have loaded training and test datasets, and we are ready to start training our first neural network!